In [10]:
%reload_azureml_ws

2020-04-01 11:51:47,769 - azureml.core.workspace - INFO - Found the config file in: /home/yuvraj/projects/AzureAworkspace/azureds/.azureml/config.json
Ready to use Azure ML 1.2.0 to work with azml-workspace
Imported workspace as ws
2020-04-01 11:51:48,990 - root - INFO - Defined global variable `ws` and `conf_catalog`


# 1. Azure Datasets

An Azure [***Dataset***](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py) represents a resource for 

- exploring,
- transforming, and
- managing 

data in AzureML.

Datasets are the recommended way to work with data, and are the primary mechanism for advanced Azure Machine Learning capabilities like data labeling and data drift monitoring.

>*** A Dataset is a reference to data in a Datastore or behind public web urls.***

## 1.1 Type of Azure Datasets 

You can create the following types of dataset:

- ***Tabular*** : The data is read from the dataset as a table. You should use this type of dataset when your data is consistently structured and you want to work with it in common tabular data structures, such as Pandas dataframes.

- ***File*** : The dataset presents a list of file paths that can be read as though from the file system. Use this type of dataset when your data is unstructured, or when you need to process the data at the file level (for example, to train a convolutional neural network from a set of image files).

# 2. Creating and Registering Tabular Datasets

## 2.1 Create Tabular Dataset
To create a tabular dataset using the SDK, use the from_delimited_files method of the Dataset.Tabular class, like this:

In [11]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds,'01_raw/*.csv'))


2020-04-01 11:51:49,005 - azureml.data.datastore_client - INFO - <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f0ff822ff60>


/home/yuvraj/anaconda3/envs/AzureDS/lib/python3.6/site-packages/_pytest/mark/structures.py:443: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


## 2.2 Register Tabular Dataset
We will register the tabular dataset as Iris:

In [12]:
# Register the tabular dataset
tab_data_set = tab_data_set.register(workspace=ws, 
                                   name='Iris Dataset',
                                   description='iris data',
                                   tags = {'format':'CSV'},
                                   create_new_version=True)

## 2.3 Creating and Registering File Datasets

To create a file dataset using the SDK, use the from_files method of the Dataset.File class, like this:

In [13]:
#Create a file dataset from the path on the datastore (this may take a short while)
file_data_set = Dataset.File.from_files(path=(default_ds, '01_raw/*.csv'))

# Get the files in the dataset
for file_path in file_data_set.to_path():
    print(file_path)

try:
    # Register the file dataset
    file_data_set = file_data_set.register(workspace=ws, 
                                       name='Iris Files Dataset',
                                       description='Iris file dataset',
                                       tags = {'format':'CSV'},
                                       create_new_version=True)
    print('Datasets registered')
except:
    print('Dataset File already registered')



2020-04-01 11:51:55,047 - azureml.core.run - INFO - Could not load the run context. Logging offline
/iris.csv
Dataset File already registered


You can view and manage datasets on the Datasets page for your workspace in Azure ML Studio. You cal also get a list of datasets from the workspace object:

In [14]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)


Datasets:
2020-04-01 11:51:59,008 - azureml.core.run - INFO - Could not load the run context. Logging offline
	 diabetes file dataset version 1
2020-04-01 11:51:59,584 - azureml.core.run - INFO - Could not load the run context. Logging offline
	 Iris Dataset version 1


# 3. Retrieving a Registered Dataset
After registering a dataset, you can retrieve it by using any of the following techniques:

- The ***datasets*** dictionary attribute of a ***Workspace*** object.
- The ***get_by_name*** or ***get_by_id*** method of the ***Dataset*** class.

Both of these techniques are shown in the following code:

In [15]:
from azureml.core import Workspace, Dataset

# Get a dataset from the workspace datasets collection
ds1 = ws.datasets['Iris Dataset']

# Get a dataset by name from the datasets class
ds2 = Dataset.get_by_name(ws, 'Iris Dataset')

2020-04-01 11:52:00,338 - azureml.core.run - INFO - Could not load the run context. Logging offline
2020-04-01 11:52:01,672 - azureml.core.run - INFO - Could not load the run context. Logging offline


In [16]:
# Display the first 20 rows as a Pandas dataframe

data = ds1.to_pandas_dataframe()

2020-04-01 11:52:01,703 - azureml.core.run - INFO - Could not load the run context. Logging offline


In [17]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [18]:
 ws.datasets.get('Iris Dataset')

2020-04-01 11:52:02,849 - azureml.core.run - INFO - Could not load the run context. Logging offline


{
  "source": [
    "('azuremlprimary', '01_raw/*.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "263d5808-7443-4572-b5f1-79a55d499163",
    "name": "Iris Dataset",
    "version": 1,
    "description": "iris data",
    "tags": {
      "format": "CSV"
    },
    "workspace": "Workspace.create(name='azml-workspace', subscription_id='797942f9-de86-4b1e-8b36-7ba7238edc7b', resource_group='aml-resources')"
  }
}